In [1]:
!pip install psycopg[binary]
!pip install sentence-transformers


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.0 MB 1.2 MB/s eta 0:00:10
   -- ------------------------------------- 0.8/12.0 MB 1.1 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/12.0 MB 1.1 MB/s eta 0:00:11
   ---- ----------------------------------- 1.3/12.0 MB 1.1 MB/s eta 0:00:10
   ----- ---------------------------------- 1.6/12.0 MB 1.1 MB/s eta 0:00:10
   ----- ---------------------------------- 1.6/12.0 MB 1.1 MB/s eta 0:00:10
   ------ --------------------------------- 1.8/12.0 MB 1.1 MB/s eta 0:00:10
   ------ --------------------------------- 2.1/12.0 MB 1.1 MB/s eta 0:00:10
   ------- --------------------------------

In [ ]:
import psycopg
from psycopg import Cursor
from sentence_transformers import SentenceTransformer
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Charger les variables d'environnement depuis le fichier .env
load_dotenv(dotenv_path="C:\\Users\\Administrateur\\Chatbot-RAG\\src\\.env")
# Variables de base de données depuis .env
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_PORT = os.getenv("DB_PORT")
# Construire la chaîne de connexion
DB_CONNECTION_STR = f"dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} host={DB_HOST} port={DB_PORT}"

# Déclarer les variables nécessaires
CONVERSATION_FILE_PATH = os.getenv("CONVERSATION_FILE_PATH", "")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
db_connection_str = "dbname=rag_chatbot user=postgres password=polyvalent00 host=localhost port=5432"

# Configuration des modèles
model_name = "all-MiniLM-L6-v2"   
model = SentenceTransformer(model_name)
VECTOR_DIM = 384  

groq_llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0, api_key=GROQ_API_KEY)

def create_conversation_list(file_path: str) -> list[str]:
    encoding = "latin-1"    
    with open(file_path, "r", encoding=encoding) as file:
        text = file.read()
        text_list = text.split("\n")
        filtered_list = [chaine.removeprefix("     ") for chaine in text_list if not chaine.startswith("<")]
        print(filtered_list)
        return filtered_list

# Fonction embeddings HuggingFace
def calculate_embeddings(corpus: str, model: SentenceTransformer) -> list[float]:
    embedding = model.encode(corpus, convert_to_tensor=False).tolist()
    return embedding    

# Convertir embedding en format pgvector
def embedding_to_pgvector_format(emb: list[float]) -> str:
    return "[" + ",".join(map(str, emb)) + "]"

# Sauvegarder embedding dans PostgreSQL
def save_embedding(corpus:str, embedding:list[float],cursor: Cursor)->None:
    embedding_formatted = embedding_to_pgvector_format(embedding)
    cursor.execute('''INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)''', 
                  (corpus, embedding_formatted)) 


def similar_corpus(input_corpus: str, db_connection_str: str) -> list[tuple[int, str, float]]:
    emb = calculate_embeddings(input_corpus, model)
    emb_formatted = embedding_to_pgvector_format(emb)
    
    results = []
    with psycopg.connect(db_connection_str) as conn:
        with conn.cursor() as cur:
            cur.execute(
                """
                SELECT id, corpus, embedding <-> %s::vector AS distance
                FROM embeddings
                ORDER BY embedding <-> %s::vector
                LIMIT 5
                """,
                (emb_formatted, emb_formatted)
            )
            rows = cur.fetchall()
            for row in rows:
                results.append((row[0], row[1], row[2]))
    return results


def generate_answer_with_groq(user_query: str, context_texts: list[str]) -> str:
    """
    Utilise LangChain Groq pour générer une réponse basée sur le contexte
    """
    # Construction du prompt avec le contexte
    context = "\n".join([f"- {text}" for text in context_texts])
    
    prompt = f"""Tu es un assistant utile qui répond aux questions en utilisant exclusivement le contexte fourni.
Si l'information n'est pas dans le contexte, dis que tu ne sais pas. Réponds en français.

Contexte:
{context}

Question: {user_query}

Réponse:"""
    
    try:
        response = groq_llm.invoke(prompt)
        return response.content
    except Exception as e:
        return f"❌ Erreur avec Groq API: {str(e)}"

# Initialisation de la base de données
with psycopg.connect(db_connection_str) as conn:
    conn.autocommit = True
    with conn.cursor() as cur:
        cur.execute("""DROP TABLE IF EXISTS embeddings""")
        cur.execute("""CREATE EXTENSION IF NOT EXISTS vector""")
        cur.execute("""CREATE TABLE IF NOT EXISTS embeddings (ID SERIAL PRIMARY KEY, 
                    corpus TEXT,
                    embedding vector(384));  
                    """)
        
        corpus_list = create_conversation_list(file_path=conversation_file_path)
   
        for corpus in corpus_list:
            embedding = calculate_embeddings(corpus=corpus, model=model) 
            save_embedding(corpus=corpus, embedding=embedding, cursor=cur)       
        conn.commit()

# BOUCLE PRINCIPALE
print("\n" + "="*60)
print("🤖 CHATBOT RAG COMPLET - Embeddings + Groq LLM")
print("="*60)

while True:
    print("\nTapez votre question (ou 'quit' pour quitter):")
    user_query = input("> ")
    print(f"🔹 Question reçue : {user_query}")
    if user_query.lower() == 'quit':
        print("Au revoir!")
        break
    
    if not user_query.strip():
        print("Veuillez entrer une question valide.")
        continue
    
    print(f"\n🔍 Recherche de contextes similaires...")
    print("-" * 50)
    
    # Étape 1: RECHERCHE (Retrieval)
    similar_results = similar_corpus(user_query, db_connection_str)
    
    if similar_results:
        context_texts = [corpus for _, corpus, _ in similar_results]
        
        print(f"📚 {len(similar_results)} contextes trouvés:\n")
        for i, (id, corpus, distance) in enumerate(similar_results, 1):
            print(f"{i}. [Distance: {distance:.4f}] {corpus}")
            print("-" * 40)
        
        # Étape 2: GÉNÉRATION (Generation)
        print("\n🤖 Génération de la réponse avec Groq...")
        print("=" * 50)
        
        answer = generate_answer_with_groq(user_query, context_texts)
        
        print(f"💬 RÉPONSE:\n{answer}")
        print("=" * 50)
        
    else:
        print("Aucun contexte trouvé dans la base de données.")

['h: U B S bonjour', "c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e", 'h: je vous écoute', "c: c'est pour", "c: e c'est pour savoir si la fac pendant l'été e a des professeurs ou des des gens qui font des stages de de perfectionnement en anglais et en espagnol", 'h: e ce serait pour vous vous souhaiteriez', 'h: non', "c: non non c'est pas pour moi", 'c: ce serait pour ma fille', 'h: oui', 'c: mais bon elle est', 'c: en seconde et', 'h: et elle souhaiterais se perfectionner', 'c: bon elle va passer en première mais', "h: en anglais ou en espagnol pendant l'été", "c: ouais c'est çà", 'h: oui alors e la fac de e de lettre et de langues se trouve à Lorient donc il faudrait plutôt  voir avec Lorient pour e savoir si ils organisent des stages mais en tout cas fac est fermée du 23 juillet au 23 août', "c: ouais par contre vous pas me m'orienter vers e", 'h: vers Lorient', 'c: e', "c: un organisme sur Vannes qui qui s'occupe de ce genre de chose", 'h: e non à priori non 